<a href="https://www.kaggle.com/code/josephfaster/video-games-forecasting-2017?scriptVersionId=144603575" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Análisis de la industria de los videojuegos de su nacimiento a 2016
- `José Francisco Lara Cárdenas`
- `jo.franciscolara@gmail.com`



# Índice
1. [Descripción del Proyecto](#descripcion-del-proyecto)


2. [Cargar datos](#cargar-datos)


3. [Visualizar dataframe y observaciones preliminares](#visualizar-dataframe-y-observaciones-preliminares)
   - [Observar data general](#observar-data-general)
   - [Encontrar valores ausentes](#encontrar-valores-ausentes)
   - [Encontrar valores duplicados](#encontrar-valores-duplicados)
   - [Encontrar valores únicos](#encontrar-valores-unicos)
   
   
4. [Preparar datos y limpieza de datos](#preparar-datos-y-limpieza-de-datos)
   - [Valores Nulos restantes](#valores_nulos_restantes)
   - [EDA / para limpiar datos](#EDA-para-limpiar-datos)
   
   
5. [Análisis de datos](#análisis-de-datos)
   - [Juegos lanzados en diferentes años](#juegos-lanzados-en-diferentes-años)
   - [Ventas por plataforma](#ventas_por_plataforma)
   - [Determinar el periodo relevante para el análisis](#determinar_el_periodo_relevante_para_el_análisis)
   - [Plataformas lideres en ventas y su evolución](#plataformas_lideres_en_ventas_y_su_evolución)
   - [Diagrama de caja para ventas globales por plataforma](#diagrama_de_caja_para_ventas_globales_por_plataforma)
   - [Efecto de las reseñas en las ventas](#efecto_de_las_reseñas_en_las_ventas)
   - [Comparar ventas de juegos en otras plataformas](#comparar_ventas_de_juegos_en_otras_plataformas)
   - [Distribución general de juegos por géneros](#Distribución_general_de_juegos_por_género)
   
   
6. [Crea un perfil de usuario para cada región](#crea_un_perfil_de_usuario_para_cada_región)
   - [Las cinco plataformas principales](#las-cinco-plataformas-principales)
   - [Los cinco generos principales para cada región y sus diferencas](#los-cinco-generos-principales-para-cada-región-y-sus-diferencias)
   - [Si las clasificaciones de ESRB afectan a las ventas en regiones individuales
](#si-las-clasificaciones-de-ESRB-afectan-a-las-ventas-en-regiones-individuales)


7. [Prueba de Hipotésis](#prueba-de-hipotesis)
   - [Resultado de las pruebas](#resultados-de-las-pruebas)
   - [Conclusiones](#conclusiones)
   - [Justificación](#justificación)
   - [Conclusión General*](#conclusión-general)


<a id="descripcion-del-proyecto"></a>
## **Descripción del Proyecto**



El objetivo del proyecto es identificar tendencias y patrones que determinen si un juego tiene éxito o no, buscado detectar proyectos prometedores y planificar campañas publicitarias.

Para ello me basare en los datos de las reseñas de usuarios y expertos, los géneros, las plataformas (por ejemplo, Xbox o PlayStation) y los datos históricos sobre las ventas de juegos que están disponibles en fuentes abiertas, teniéndolo en un data set llamado game_df.


<a id="cargar-datos"></a>
## Cargar datos


In [ ]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re

<a id="visualizar-dataframe-y-observaciones-preliminares"></a>
## Visualizar dataframe y observaciones preliminares


<a id="observar-data-general"></a>
### Observar data general


In [ ]:
!ls /kaggle/input/games-df-2016


In [ ]:
# Mostrar base de datos

games_df = pd.read_csv('/kaggle/input/games-df-2016/games.csv')
display(games_df) 
# Mostrar información general
print(games_df.info())

**Observaciones:**

- `Name`: Nombre del juego
- `Platform`: Plataforma en la que se juega el videojuego (e.g., Wii, NES)
- `Year_of_Release`: Año de lanzamiento del juego
- `Genre`: Género del juego
- `NA_sales`: Ventas en Norteamérica (en millones de dolares)
- `EU_sales`: Ventas en Europa (en millones de dolares)
- `JP_sales`: Ventas en Japón (en millones de dolares)
- `Other_sales`: Ventbas en otras regiones (en millones de dolares)
- `Critic_Score`: Puntuación dada por los críticos
- `User_Score`: Puntuación dada por los usuarios
- `Rating`: Clasificación ESRB

****
- Los nombres de las columnas no estan en formato snake_case.
****
La tabla tiene un total de 16675 filas, los tipos de datos son:

- `name` (object): 16713 valores no nulos.
- `platform` (object): todos los valores están presentes.
- `year_of_release` (float64): 16446 valores no nulos. Es preferible convertir esta columna a un tipo de dato entero (int) después de manejar los valores nulos.
- `genre` (object): 16713 valores no nulos.
- `na_sales`, `eu_sales`, `jp_sales`, `other_sales` y `critic_score` (float64): tienen sus respectivos números de valores no nulos.
- `user_score` (object): 10014 valores no nulos. Esta columna debería ser de tipo float, pero parece que hay valores no numéricos (como "tbd") que impiden la conversión directa.
- `rating` (object): 9949 valores no nulos.



<a id="encontrar-valores-ausentes"></a>
### Encontrar valores ausentes


In [ ]:
# Encontrar valores ausentes
NaN = games_df.isna().sum()
display(NaN)

**Comentario:** 
El conjunto de datos presenta valores ausentes en las siguientes columnas:

1. **Name**: 2 valores ausentes. Esta es una cantidad insignificante en relación con el tamaño total del conjunto de datos. Podría deberse a errores en la entrada de datos o a problemas durante la recopilación de datos.
2. **Year_of_Release**: 269 valores ausentes. Es crucial determinar el año de lanzamiento de un juego para realizar análisis temporales o identificar tendencias a lo largo del tiempo.
3. **Genre**: 2 valores ausentes. Al igual que con el nombre, es una cantidad mínima. Sin embargo, el género es una característica importante para clasificar y analizar juegos.
4. **Critic_Score**: 8,578 valores ausentes. Esta es una cantidad considerable y sugiere que no todos los juegos en el conjunto de datos fueron revisados o calificados por críticos.
5. **User_Score**: 6,701 valores ausentes. Al igual que con `Critic_Score`, no todos los juegos recibieron puntuaciones de los usuarios. Además, este número incluye aquellos juegos con puntuación "tbd".
6. **Rating**: 6,766 valores ausentes. La clasificación ESRB es esencial para determinar el grupo demográfico al que está dirigido un juego. La falta de estas clasificaciones podría deberse a que algunos juegos no fueron sometidos al proceso de clasificación de ESRB o no se recopiló esa información específica.

La presencia de valores ausentes, especialmente en columnas clave como `Critic_Score`, `User_Score` y `Rating`, puede afectar la calidad y precisión de cualquier análisis posterior. Por lo tanto, será esencial decidir cómo tratar estos valores ausentes: ya sea eliminándolos, imputándolos o dejándolos según el tipo de análisis que se  realice más adelante.

<a id="encontrar-valores-duplicados"></a>
### Encontrar valores duplicados


In [ ]:
#Encontrar valores duplicados
display(games_df.duplicated().sum())

**Comentario:** 

No hay filas duplicadas en el conjunto de datos. Esto es excelente ya que indica que el conjunto de datos es limpio en términos de entradas repetidas. La ausencia de duplicados facilita y mejora la precisión de cualquier análisis posterior, ya que no se inflarán los resultados debido a entradas redundantes. 

<a id="encontrar-valores-unicos"></a>
#### Encontrar valores únicos


In [ ]:
#Encontrar valores únicos para el análisis
unique_values_original = {}
for column in games_df.columns:
    unique_values_original[column] = games_df[column].unique()

print(unique_values_original)

**Observaciones**
1. **Diversidad de Juegos**: El conjunto de datos contiene una amplia gama de videojuegos, cada uno identificado por su nombre único.
2. **Plataformas Variadas**: Hay 31 plataformas diferentes registradas, desde consolas clásicas como NES hasta más modernas como PS4 y X360.
3. **Rango Temporal**: Los datos abarcan juegos lanzados desde 1980 hasta 2016. Sin embargo, hay algunos valores nulos en el año de lanzamiento que pueden requerir tratamiento.
4. **Géneros**: El conjunto de datos categoriza los juegos en varios géneros, lo que podría ser útil para análisis relacionados con tendencias de género.
5. **Puntuaciones**: Tanto las puntuaciones de los críticos como de los usuarios presentan valores nulos. Además, la columna `User_Score` contiene valores "tbd", lo que indica que la puntuación aún no ha sido determinada.
6. **Clasificaciones ESRB**: Aunque estas clasificaciones son esenciales para entender el grupo demográfico objetivo de un juego, hay valores nulos que podrían complicar el análisis basado en la clasificación.

Para avanzar, es esencial abordar los valores nulos y "tbd" antes de realizar análisis más profundos. Dada la naturaleza del conjunto de datos, hay muchas oportunidades para investigar tendencias, preferencias y el éxito de diferentes juegos según diversas métricas (como ventas, puntuaciones, etc.).

<a id="preparar-datos-y-limpieza-de-datos"></a>
## Preparar datos y limpieza de datos

Lo paso por hacer son:

- 1 Reemplazar los nombres de las columnas para que estén en minúsculas (aunque ya están en snake_case, aseguraremos que estén en minúsculas).
- 2 Convertir los datos a los tipos necesarios.
- 3 Describir y justificar los cambios de tipo de datos realizados.
- 4 Tratar los valores ausentes y explicar el método utilizado.
- 5 Abordar la abreviatura "TBD" en la columna correspondiente.
- 6 Calcular las ventas totales para cada juego y añadir estos valores en una nueva columna.

In [ ]:
# 1. Reemplazar los nombres de las columnas
games_df.columns = games_df.columns.str.lower().str.replace(" ", "_")

# 2. Convertir los datos a los tipos necesarios
# Convertir 'year_of_release' a tipo entero después de reemplazar los NaNs con un valor temporal
games_df['year_of_release'].fillna(0, inplace=True)  # Reemplazando NaNs con 0 temporalmente
games_df['year_of_release'] = games_df['year_of_release'].astype(int)

# Reemplazar 'tbd' con NaN en 'user_score' y convertir a float
games_df['user_score'].replace('tbd', np.nan, inplace=True)
games_df['user_score'] = games_df['user_score'].astype(float)

# 3. Describir y justificar los cambios de tipo de datos realizados
data_types_changes = {
    "year_of_release": "Convertido a int64. Los años son valores discretos y no necesitamos decimales.",
    "user_score": "Convertido a float64. Para realizar cálculos numéricos en futuros análisis. Los valores 'tbd' se reemplazaron con NaN."
}

# 4. Tratar los valores ausentes
# Para 'year_of_release', hemos reemplazado NaNs con 0 temporalmente. Podemos considerar reemplazar estos ceros con NaN nuevamente 
# para representar la ausencia de datos de manera más clara.
games_df['year_of_release'].replace(0, np.nan, inplace=True)

# 5. Abordar la abreviatura "TBD" en la columna correspondiente (ya lo hicimos anteriormente).

# 6. Calcular las ventas totales para cada juego y añadir estos valores en una nueva columna
games_df['total_sales'] = games_df[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)

data_types_changes, games_df.info()


display(games_df)
display(data_types_changes)
display(games_df.info())

**Resumen**
1. Se han reemplazado los nombres de las columnas con minúsculas y en formato snake_case.
2. Se han convertido los datos a los tipos necesarios:
    - La columna `year_of_release` se ha convertido a tipo `int64` porque los años son valores discretos y no necesitamos decimales.
    - La columna `user_score` se ha convertido a tipo `float64` para realizar cálculos numéricos en futuros análisis. También hemos reemplazado los valores 'tbd' con NaN.
3. He tratado los valores ausentes en `year_of_release`, reemplazando los NaNs con 0 temporalmente. Luego, se reemplazaron estos ceros con NaN para representar la ausencia de datos de manera más clara.
4. Calcule las ventas totales para cada juego y añadimos estos valores en una nueva columna `total_sales`.


<a id="valores_nulos_restantes"></a>
### Valores nulos restantes


In [ ]:
# Mostrando un resumen de los valores ausentes en cada columna
missing_values = games_df.isnull().sum()

missing_values_percentage = (games_df.isnull().sum() / len(games_df)) * 100

print('Valores ausentes')
display(missing_values)
print()
print('Valores ausentes en %')
display(missing_values_percentage)


1. Para `name` y `genre`, imputaré los valores ausentes usando la moda.

In [ ]:
# Imputando valores ausentes en 'name' y 'genre' usando la moda

# Moda para 'name' y 'genre'
name_mode = games_df['name'].mode()[0]
genre_mode = games_df['genre'].mode()[0]

# Rellenando NaNs con la moda
games_df['name'].fillna(name_mode, inplace=True)
games_df['genre'].fillna(genre_mode, inplace=True)

# Verificando si aún hay valores nulos en 'name' y 'genre'
name_genre_nulls_after = games_df[['name', 'genre']].isnull().sum()

name_genre_nulls_after


Ahora lo que se hará para tratar de limpiar esos datos es lo siguiente:

In [ ]:
# Modificando la función para manejar valores no string
def extract_year_from_name(name):
    # Asegurando que el valor de entrada es una cadena
    if isinstance(name, str):
        # Usando regex para identificar años en el formato de 4 dígitos en el nombre
        match = re.search(r'(\d{4})', name)
        return int(match.group()) if match else np.nan
    return np.nan

# Aplicando la función corregida para extraer años de los nombres
games_df['imputed_year'] = games_df['name'].apply(extract_year_from_name)

# Rellenar NaNs en 'year_of_release' con 'imputed_year' donde sea posible
games_df['year_of_release'].fillna(games_df['imputed_year'], inplace=True)

# Eliminando la columna temporal 'imputed_year'
games_df.drop('imputed_year', axis=1, inplace=True)

# 2. Imputar valores ausentes en 'year_of_release' basado en juegos con el mismo nombre
games_without_year = games_df[games_df['year_of_release'].isnull()]
for index, row in games_without_year.iterrows():
    # Buscar juegos con el mismo nombre que tengan un año registrado
    same_name = games_df[(games_df['name'] == row['name']) & (~games_df['year_of_release'].isnull())]
    if not same_name.empty:
        year = same_name['year_of_release'].iloc[0]
        games_df.at[index, 'year_of_release'] = year
        
# Función para imputar año basado en nombres que terminan en dos dígitos
def impute_year_from_two_digit(name, current_year):
    # Asegurando que el valor de entrada es una cadena
    if isinstance(name, str):
        # Usando regex para identificar años en el formato de 2 dígitos al final del nombre
        match = re.search(r'(\d{2})$', name)
        if match:
            year = int(match.group())
            # Asumiendo que cualquier juego con un año de 00-21 fue lanzado en 2000-2021
            # Y cualquier juego con un año de 22-99 fue lanzado en 1922-1999
            return year + 2000 if year <= 21 else year + 1900
    return current_year

# Aplicando la función para imputar años en los registros que aún tienen NaN en 'year_of_release'
games_df['year_of_release'] = games_df.apply(
    lambda row: impute_year_from_two_digit(row['name'], row['year_of_release']), axis=1
)
missing_values = games_df.isnull().sum()
display(missing_values)

percent = 135/16446 *100
percent

Lo realizado en el codigo anterior fue complejo pero aquí esta el resumen:
1. **Imputación a partir del nombre del juego**: 
   - Extraje años potenciales de los nombres de los juegos. Por ejemplo, para juegos con nombres como "FIFA 2005", tome "2005" como el año de lanzamiento y lo use para llenar los valores faltantes. 
   - También considere juegos que tenían una nomenclatura de dos dígitos referente a los años, como "99" para "1999".

2. **Imputación basada en juegos con nombres idénticos**:
   - Busque juegos que tuvieran el mismo nombre pero diferentes plataformas. Si uno de estos juegos tenía un año de lanzamiento especificado, usé ese año para imputar los valores faltantes de juegos con el mismo nombre.

Al final quedaron 135 valores nulos, que los dejare como nulos para que no afecten el análisis más adelante, esos datos solo representan un .82% del total de los datos en la columna, una cifra maginal.


<a id="EDA-para-limpiar-datos"></a>

### EDA 

In [ ]:
# Visualizar la distribución de los datos con un histograma.
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))

# Histogramas
sns.histplot(games_df['critic_score'], bins=30, ax=axes[0, 0], color='skyblue', kde=True)
axes[0, 0].set_title('Distribución de critic_score')
sns.histplot(games_df['user_score'], bins=30, ax=axes[0, 1], color='salmon', kde=True)
axes[0, 1].set_title('Distribución de user_score')

# Boxplots
sns.boxplot(x=games_df['critic_score'], ax=axes[1, 0], color='skyblue')
axes[1, 0].set_title('Boxplot de critic_score')
sns.boxplot(x=games_df['user_score'], ax=axes[1, 1], color='salmon')
axes[1, 1].set_title('Boxplot de user_score')

plt.tight_layout()
plt.show()

# Calcular la media y la mediana para comparar.
media_critic = games_df['critic_score'].mean()
mediana_critic = games_df['critic_score'].median()
media_user = games_df['user_score'].mean()
mediana_user = games_df['user_score'].median()

print(f"Critic_Score - Media: {media_critic}, Mediana: {mediana_critic}")
print(f"User_Score - Media: {media_user}, Mediana: {mediana_user}")

# Verificar la cantidad de valores nulos en estas columnas
remaining_nulls = games_df[['critic_score', 'user_score']].isnull().sum()
print(remaining_nulls)

remaining_nulls_total = games_df.isnull().sum()
print(remaining_nulls_total)



**Comentario:**

- **Critic_Score**: 
  - La distribución de las calificaciones de los críticos parece sesgada hacia la izquierda, lo que indica que hay más juegos con calificaciones más altas que bajas.
  - El diagrama de caja revela algunos valores atípicos en el extremo inferior, lo que sugiere que hay algunos juegos que han sido puntuados significativamente más bajos que la mayoría.
  - La media y la mediana están relativamente cerca una de la otra, pero la mediana es ligeramente mayor, lo que refuerza la observación del sesgo hacia la izquierda.

- **User_Score**: 
  - La distribución de las calificaciones de los usuarios parece un poco más uniforme en comparación con la de los críticos, aunque todavía presenta un ligero sesgo hacia la izquierda.
  - El diagrama de caja para las calificaciones de los usuarios muestra una dispersión más amplia en comparación con las calificaciones de los críticos, y también tiene algunos valores atípicos en ambos extremos.
  - Al igual que con las calificaciones de los críticos, la media y la mediana de las calificaciones de los usuarios están cerca una de la otra, pero la mediana es ligeramente mayor.

Ambas distribuciones, tanto para las calificaciones de los críticos como para las de los usuarios, muestran una tendencia hacia calificaciones más altas. Sin embargo, los usuarios tienen una mayor dispersión en sus calificaciones en comparación con los críticos profesionales. Es esencial tener en cuenta estos patrones y tendencias al analizar más a fondo o al tomar decisiones basadas en estas calificaciones.


Solo falta limpiar rating.

Los valores únicos de la columna "Rating" son:

1. **E**: "Everyone" (Para todos)
2. **M**: "Mature" (Maduro - Para mayores de 17 años)
3. **T**: "Teen" (Adolescentes - A partir de 13 años)
4. **E10+**: "Everyone 10 and older" (Para todos mayores de 10 años)
5. **K-A**: "Kids to Adults" (Este es un antiguo rating que fue reemplazado por "E". Esencialmente, significa lo mismo que "E")
6. **AO**: "Adults Only" (Solo para adultos - 18 años y más)
7. **EC**: "Early Childhood" (Infancia temprana - Para niños de 3 años en adelante)
8. **RP**: "Rating Pending" (Calificación pendiente)

El valor NaN representa la ausencia de datos en esta columna.

Cada una de estas calificaciones es proporcionada por la ESRB (Entertainment Software Rating Board) y ayuda a los consumidores a identificar el contenido del juego y determinar si es apropiado para el jugador.

- Una hipotésis que me surge antes de previzualizar los datos sobres los valores nulos, es posible que Los juegos más antiguos, especialmente aquellos lanzados antes de la creación del sistema de clasificación ESRB en 1994, naturalmente no tendrían una clasificación ESRB.

- Otra hipotésis puede ser que la ESRB al ser organización estadounidense, los juegos lanzados en otras regiones, como Japón o Europa, podrían no tener una clasificación ESRB.

In [ ]:
# Filtrando las filas donde el rating es ausente
missing_rating_df = games_df[games_df['rating'].isna()]

missing_rating_df.head()  # Mostrando las primeras filas del dataframe resultante


Al observar la tabla, se ve como coniciden las columnas `user_score`, `critic_score` y `rating` en muchas filas con valores nulos, esto me parece interesante, haré un análisis más profundo más adelante.

In [ ]:
# Verificando la cantidad de valores nulos en 'User_Score' y 'Critic_Score' para juegos sin clasificación
missing_user_score = games_df['user_score'].isna().sum()
missing_critic_score = games_df['critic_score'].isna().sum()

missing_user_score, missing_critic_score, len(missing_rating_df)


Efectivamente

- De los 6,766 juegos sin clasificación ESRB ("Rating"):
  - 6,630 juegos también carecen de "User_Score".
  - 6,683 juegos también carecen de "Critic_Score".

Esto indica que la gran mayoría de juegos sin clasificación ESRB también carecen de puntuaciones de usuarios y críticos.

**Análisis:**

1. **Recopilación de Datos**: Podría ser que la fuente de datos no tenía información disponible tanto para las clasificaciones como para las puntuaciones de ciertos juegos, lo que indica un posible problema en la recopilación de datos o en la fuente de datos original.
  
2. **Juegos menos populares o antiguos**: Es posible que ciertos juegos, especialmente aquellos que son más antiguos o menos populares, no hayan sido revisados por críticos ni clasificados por la ESRB. También es posible que no hayan recibido suficientes reseñas de usuarios para tener una puntuación significativa.

3. **Juegos lanzados en regiones específicas**: Como se mencionó anteriormente, algunos juegos que se lanzaron exclusivamente en regiones fuera de América del Norte podrían no haber sido clasificados por la ESRB y, por lo tanto, también podrían carecer de reseñas en fuentes que se centran principalmente en el mercado norteamericano.

En resumen, la coincidencia entre la falta de "Rating" y la falta de puntuaciones sugiere que hay un subconjunto de juegos en la base de datos que, por diversas razones, no han sido revisados ni clasificados ampliamente. Esta observación destaca la importancia de entender y considerar las limitaciones de los datos al realizar análisis y sacar conclusiones.

**Otro análisis que voy a probar son correlaciones y distribuciones:**
1. Crear una columna booleana que indique si "Rating" es nulo o no.
2. Analizar las correlaciones entre esta columna y otras columnas numéricas.
3. Examinar las distribuciones y tendencias en columnas categóricas (como "Platform" o "Genre") para juegos con y sin clasificación.

In [ ]:
# Creando la columna 'Rating_is_null' con la corrección del nombre de la columna
games_df['Rating_is_null'] = games_df['rating'].isna().astype(int)

# Filtrar el DataFrame para incluir solo columnas numéricas antes de calcular la correlación
numeric_columns = games_df.select_dtypes(include=[float, int])

# Calculando las correlaciones con la columna 'Rating_is_null'
correlation_with_rating_null_corrected = numeric_columns.corr()['Rating_is_null'].drop('Rating_is_null')
correlation_with_rating_null_corrected


Aquí están las correlaciones entre la ausencia de clasificación ESRB ("Rating") y otras columnas numéricas:

- **Year_of_Release**: -0.198
  - Hay una correlación negativa moderada entre la ausencia de clasificación y el año de lanzamiento. Esto sugiere que los juegos más antiguos tienen más probabilidades de no tener una clasificación ESRB, lo que es coherente con el hecho de que la ESRB se creó en 1994. 
- **NA_sales**: -0.081, **EU_sales**: -0.089, **Other_sales**: -0.110
  - Hay una pequeña correlación negativa entre la ausencia de clasificación y las ventas en América del Norte, Europa y otras regiones. Esto podría indicar que los juegos sin clasificación ESRB tienden a vender menos en estas regiones, posiblemente debido a la importancia de la clasificación para los consumidores.
- **JP_sales**: 0.125
  - Hay una pequeña correlación positiva con las ventas en Japón. Esto puede indicar que en Japón, la clasificación ESRB es menos crucial para las ventas, quizás debido a la presencia de sistemas de clasificación locales como CERO.
- **Critic_Score**: -0.0026
  - No hay una correlación significativa entre la ausencia de clasificación y la puntuación del crítico.



Para estudiar más a detalle los datos haré un análisis segemnetado de de los datos para ver sus distribuciones
**Análisis Segmentado**.

En este análisis dividó el conjunto de datos en segmentos y analizó cada segmento de manera independiente. En este caso, dividire el conjunto de datos en dos segmentos:

1. Juegos con una clasificación ESRB conocida.
2. Juegos sin una clasificación ESRB (es decir, valores ausentes).

Despues de haber hecho lo anterior, podría realizar u análisis separados para cada segmento. Esto me permitiría identificar tendencias y patrones específicos de cada segmento sin que la falta de datos en uno de los segmentos sesgue o distorsione los resultados.

Procedimiento:

1. Crearemos dos DataFrames separados: uno para juegos con una clasificación ESRB y otro para juegos sin clasificación.
2. Analizaremos cada segmento por separado, observando las características y distribuciones clave.
3. Podemos comparar y contrastar los resultados de cada segmento para obtener insights.

In [ ]:
# Segmentando el conjunto de datos en juegos con y sin clasificación ESRB
segmento_df = pd.read_csv('/kaggle/input/games-df-2016/games.csv')
games_with_rating = segmento_df[segmento_df['Rating'].notna()]
games_without_rating = segmento_df[segmento_df['Rating'].isna()]

# Mostrando el tamaño de cada segmento
len(games_with_rating), len(games_without_rating)


El conjunto de datos se ha segmentado en:

1. \( 9949 \) juegos con una clasificación ESRB conocida.
2. \( 6766 \) juegos sin una clasificación ESRB.

Con estos segmentos, podemos realizar análisis específicos para cada grupo. Por ejemplo, podríamos explorar las distribuciones de género, plataforma, puntuaciones de críticos y usuarios, etc., para cada segmento y compararlas.



La única solución que se me ocurre es crear una categoría "desconocida". Puedo imputar todos los valores ausentes con una categoría "desconocida" o "no clasificado". Esta es una forma directa de abordar los valores ausentes y me permitirá mantener todas las filas del conjunto de datos.


In [ ]:
# Imputando los valores ausentes en 'rating' con "Unknown"
games_df['rating'].fillna('Unknown', inplace=True)

# Verificando que no haya valores nulos en 'rating'
missing_values_rating = games_df['rating'].isnull().sum()
missing_values_rating

# Verificando si se han tratado todos los valores nulos en la columna 'rating'
games_df['rating'].value_counts()



Ahora, la columna "rating" no tiene valores nulos. Todos los valores ausentes han sido reemplazados por la etiqueta "Unknown" 

In [ ]:
# Comprobando valores nulos
remaining_nulls = games_df.isnull().sum()
remaining_nulls

La base de datos ha sido limpiada adcuadamente, ahora se podrá trabajar con ella.

<a id="análisis-de-datos"></a>

## Análisis de datos

<a id="juegos-lanzados-en-diferentes-años"></a>

### Juegos lanzados en diferentes años

Vamos a ver cuántos juegos fueron lanzados en diferentes años y si los datos de cada período son significativos.

In [ ]:
# Cantidad de juegos lanzados por año
games_per_year = games_df.groupby('year_of_release').size()

# Gráfica de juegos lanzados por año
plt.figure(figsize=(15,7))
games_per_year.plot(kind='bar', color='skyblue')
plt.title('Juegos lanzados por año')
plt.xlabel('Año de lanzamiento')
plt.ylabel('Número de juegos')
plt.show()





In [ ]:
# Filtrando juegos lanzados antes de 1994 y verificando la cantidad que no tienen clasificación de la ESRB
total_games_before_1994 = len(games_df[games_df['year_of_release'] < 1994])
print(total_games_before_1994)

La gráfica muestra la cantidad de juegos lanzados cada año. Podemos observar lo siguiente:

- Antes de 1995, el número de juegos lanzados anualmente era relativamente bajo.

- La industria experimentó un auge entre 2000 y 2010, alcanzando su punto máximo alrededor de 2008-2009.

- Después de 2010, hubo una disminución en la cantidad de juegos lanzados anualmente, lo que podría deberse a múltiples razones, como la consolidación de la industria, el aumento de los costos de desarrollo o el auge de otros tipos de entretenimiento digital.

- También se puede observar como existen un conjunto de datos particulares, los cuáles sus años son menores a 1994 que es cuando se creo la ESBR



<a id="ventas_por_plataforma"></a>

### Ventas por plataforma

observemos cómo varían las ventas de una plataforma a otra. Elegire las plataformas con las mayores ventas totales y construire una distribución basada en los datos de cada año. También buscare plataformas que solían ser populares pero que ahora no tienen ventas para determinar cuánto tiempo tardan generalmente las nuevas plataformas en aparecer y las antiguas en desaparecer.

In [ ]:
# Ventas totales por plataforma
platform_sales = games_df.groupby('platform')['total_sales'].sum().sort_values(ascending=False)

# Top 10 plataformas con las mayores ventas totales
top_platforms = platform_sales.head(10)

# Gráfica de ventas totales por plataforma para las top 10 plataformas
plt.figure(figsize=(15,7))
top_platforms.plot(kind='bar', color='lightcoral')
plt.title('Top 10 plataformas con las mayores ventas totales')
plt.xlabel('Plataforma')
plt.ylabel('Ventas totales (en millones)')
plt.show()

# Distribución de ventas por año para estas plataformas
top_platforms_list = top_platforms.index.tolist()

plt.figure(figsize=(15,7))
for platform in top_platforms_list:
    yearly_sales = games_df[games_df['platform'] == platform].groupby('year_of_release')['total_sales'].sum()
    yearly_sales.plot(label=platform)

plt.title('Distribución de ventas por año para las top 10 plataformas')
plt.xlabel('Año')
plt.ylabel('Ventas totales (en millones)')
plt.legend()
plt.show()


**Observaciones:**

Ventas totales por plataforma:

- Las plataformas con las mayores ventas totales son PS2, X360, PS3, Wii, DS, PS, PS4, GBA, PSP y PC.
- La PS2 lidera con una considerable ventaja en ventas totales, seguida de cerca por X360 y PS3.

Distribución de ventas por año para las principales plataformas:

- La PS2 tuvo su auge en ventas alrededor del año 2000 y empezó a declinar después de 2005.
- Las plataformas X360 y PS3 muestran picos similares alrededor de 2005-2015.
- La Wii tuvo un pico significativo alrededor de 2006-2010.
- La PS4, una plataforma más reciente, muestra un aumento en ventas a partir de 2013.

De la gráfica de distribución de ventas por año, podemos observar que la mayoría de las plataformas tienen un ciclo de vida de aproximadamente 10 años desde su introducción hasta su declive. Algunas plataformas, como la PS2, tuvieron una longevidad particularmente impresionante en términos de ventas.

In [ ]:
# Crear un DataFrame para las ventas anuales de las 10 principales plataformas
heatmap_data = games_df[games_df['platform'].isin(top_platforms_list)].pivot_table(
    index='year_of_release', 
    columns='platform', 
    values='total_sales', 
    aggfunc='sum'
)

# Ordenar las columnas según las ventas totales
heatmap_data = heatmap_data[top_platforms_list]

# Mostrar el heatmap
plt.figure(figsize=(15,10))
sns.heatmap(heatmap_data, cmap='YlGnBu', linewidths=0.5, annot=True, fmt=".1f")
plt.title('Ventas anuales de las 10 principales plataformas')
plt.show()


El heatmap muestra las ventas anuales de las 10 principales plataformas. Las áreas más oscuras representan mayores ventas en ese año específico para la plataforma correspondiente.

1. **Plataformas Antiguas**: Las plataformas como el NES y el GB tuvieron ventas significativas en sus primeros años, pero eventualmente desaparecieron conforme surgieron nuevas tecnologías y plataformas.
  
2. **Picos de Ventas**: Las plataformas PS2, X360, y Wii muestran picos claros en ventas, lo que indica años de máxima popularidad. Por ejemplo, el PS2 tuvo su apogeo en la primera mitad de la década de 2000, mientras que el X360 y el Wii tuvieron picos en la segunda mitad de la misma década.

3. **Desvanecimiento de Plataformas**: Algunas plataformas, como el DS y el PS3, muestran un desvanecimiento gradual en ventas. Esto podría deberse a la llegada de sucesores (como el 3DS en el caso del DS) o a la saturación del mercado.

4. **Plataformas Recientes**: El PS4 muestra un aumento en ventas a partir de su lanzamiento en 2013, lo que indica su crecimiento en popularidad en años más recientes.

5. **Consistencia en Ventas**: El PC, aunque no tiene los picos dramáticos de algunas consolas, muestra una consistencia en ventas a lo largo de los años. Esto sugiere una base de usuarios estable y la naturaleza menos cíclica del mercado de juegos de PC en comparación con las consolas.

6. **Ausencia de Datos**: La ausencia de celdas coloreadas en años recientes para algunas plataformas indica que esas plataformas ya no generan ventas significativas o han sido reemplazadas por tecnologías más recientes.

    

<a id="determinar_el_periodo_relevante_para_el_análisis"></a>
### Determinar el período relevante para el análisis
Para predecir las tendencias de 2017, debemos trabajar con datos que sean lo más recientes y relevantes posible. Dado que la mayoría de las plataformas tienen un ciclo de vida de unos 10 años, podemos considerar los datos de la última década (2007-2016) como los más relevantes para nuestro análisis. Sin embargo, para ser más precisos, podríamos considerar un rango más reciente, como 2010-2016, para nuestro análisis.


In [ ]:
# Filtrando el dataframe para considerar juegos lanzados desde 2010 en adelante
games_recent = games_df[games_df['year_of_release'] >= 2012]

# Resumen de la información del dataframe filtrado
games_recent.info()


Hemos filtrado nuestro conjunto de datos para considerar solo los juegos lanzados desde 2010 en adelante. Ahora tenemos  2.952 
entradas en nuestro dataframe, todas relevantes para el período 2012-2016.

<a id="plataformas_lideres_en_ventas_y_su_evolución"></a>
### Plataformas líderes en ventas y su evolución

Ahora, identificaremos las plataformas líderes en ventas y observaremos cuáles están creciendo y cuáles están disminuyendo. Esto nos ayudará a elegir plataformas potencialmente rentables para el futuro cercano.

In [ ]:
# Ventas totales por plataforma para el período 2013-2016
platform_sales_recent = games_recent.groupby('platform')['total_sales'].sum().sort_values(ascending=False)

# Gráfica de ventas totales por plataforma para el período 2013-2016
plt.figure(figsize=(15,7))
platform_sales_recent.plot(kind='bar', color='lightgreen')
plt.title('Ventas totales por plataforma (2013-2016)')
plt.xlabel('Plataforma')
plt.ylabel('Ventas totales (en millones)')
plt.show()


Las plataformas PS4, PS3, XOne y 3DS lideran las ventas en este período, con la PS4 y XOne mostrando cifras especialmente altas, lo que indica que podrían ser plataformas potencialmente rentables.

In [ ]:
# Obtiene las top plataformas en términos de ventas totales para el período 2013-2016
top_platforms_recent_list = platform_sales_recent.index.tolist()


In [ ]:
# Filtrar el dataframe para considerar juegos lanzados hasta 2016 (inclusive)
games_recent = games_df[(games_df['year_of_release'] >= 2013) & (games_df['year_of_release'] <= 2016)]

# Distribución de ventas por año para estas plataformas
plt.figure(figsize=(15,7))
for platform in top_platforms_recent_list:
    yearly_sales_recent = games_recent[games_recent['platform'] == platform].groupby('year_of_release')['total_sales'].sum()
    yearly_sales_recent.plot(label=platform, linewidth=2)

plt.title('Distribución de ventas por año (2013-2016)')
plt.xlabel('Año')
plt.ylabel('Ventas totales (en millones)')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()


no se guardo que raro??

**Gráfica de Ventas Totales por Plataforma (2013-2016)**
1. **Líderes en Ventas**: Las plataformas PS4, PS3 y XOne dominan claramente en ventas totales durante este período. Específicamente, la PS4 lidera por un margen significativo, lo que indica su dominio en el mercado de videojuegos en estos años.
2. **Seguidores Cercanos**: 3DS y X360 también tienen una presencia considerable, aunque sus ventas son menores en comparación con las tres plataformas líderes mencionadas anteriormente.
3. **Plataformas Menos Populares**: Plataformas como WiiU, PC y PSV muestran ventas significativamente menores en este período, lo que sugiere que no fueron tan populares o dominantes en el mercado durante estos años.

**Distribución de Ventas por Año (2013-2016)**
1. **PS4**: Esta plataforma muestra un impresionante crecimiento desde su lanzamiento en 2013, alcanzando su punto máximo en 2015. Sin embargo, hay una ligera disminución en 2016, lo que podría indicar una saturación del mercado o la aparición de competencia.
2. **PS3**: Aunque tuvo ventas sustanciales en 2013, ha experimentado una disminución constante año tras año. Esto es coherente con la transición de los jugadores de la PS3 a la PS4.
3. **XOne**: Esta plataforma ha mostrado un crecimiento constante desde su lanzamiento en 2013, aunque, al igual que la PS4, también experimenta una disminución en 2016.
4. **3DS**: A pesar de ser una de las principales plataformas en ventas totales, ha estado en declive constante desde 2013.
5. **X360**: Esta plataforma ha experimentado un declive constante, similar a la PS3, lo que sugiere que los jugadores están migrando a plataformas más recientes.
6. **Otras Plataformas**: Plataformas como WiiU, PC y PSV muestran tendencias estables o en declive, sin picos significativos.

**Conclusiones Generales**:
- Las plataformas **PS4** y **XOne** se destacan como las más prometedoras en términos de crecimiento y rentabilidad para el futuro cercano.
- Las plataformas más antiguas, como **PS3** y **X360**, están en declive, lo que indica que el mercado está cambiando hacia tecnologías más recientes.
- Mientras que algunas plataformas como **3DS** y **PC** tienen una base de usuarios estable, no muestran el mismo nivel de crecimiento que las líderes del mercado.

<a id="diagrama_de_caja_para_ventas_globales_por_plataforma"></a>

### Diagrama de caja para ventas globales por plataforma

Ahora, crearemos un diagrama de caja para las ventas globales de todos los juegos, desglosados por plataforma, para entender las diferencias en las ventas y las ventas promedio en las diversas plataformas.

In [ ]:
# Diagrama de caja para las ventas globales por plataforma
plt.figure(figsize=(18,10))
sns.boxplot(x="platform", y="total_sales", data=games_recent, palette="pastel")
plt.title('Distribución de ventas globales por plataforma (2013-2016)')
plt.xlabel('Plataforma')
plt.ylabel('Ventas totales (en millones)')
plt.ylim(0, 2.5)  # Limitamos el eje y para una mejor visualización
plt.xticks(rotation=45)
plt.show()


**Observaciones:**

El diagrama de caja nos da una idea de cómo se distribuyen las ventas globales para diferentes plataformas:

1. Mediana de ventas: Plataformas como PS4, XOne y WiiU tienen medianas más altas en comparación con otras, lo que indica que el juego promedio en estas plataformas tiende a vender más.
2. Dispersión de ventas: La PS4 y la X360 muestran una amplia dispersión de ventas, indicando que hay juegos que venden muchísimo, pero también hay muchos que no venden tanto.
3. Outliers: La mayoría de las plataformas tienen varios outliers, lo que indica que hay ciertos juegos que son excepcionalmente populares y tienen ventas muy altas en comparación con otros juegos en la misma plataforma.

En resumen, si bien ciertas plataformas como la PS4 y la XOne tienen un promedio de ventas más alto, también hay una gran variabilidad en las ventas dentro de cada plataforma. Algunos juegos son éxitos de ventas, mientras que otros no logran el mismo nivel de éxito.

<a id="efecto_de_las_reseñas_en_las_ventas"></a>
### Efecto de las reseñas en las ventas

Ahora, observaremos cómo las reseñas de usuarios y críticos afectan las ventas. Tomaremos la PS4 como plataforma de ejemplo y crearemos un gráfico de dispersión para visualizar la relación. Calculare también la correlación entre las reseñas y las ventas.

In [ ]:
# Filtrar datos para la plataforma PS4 usando el dataset reciente
ps4_data_recent = games_recent[games_recent['platform'] == 'PS4']

# Gráfico de dispersión para critic_score vs. total_sales
plt.figure(figsize=(14,6))
plt.scatter(ps4_data_recent['critic_score'], ps4_data_recent['total_sales'], color='purple', alpha=0.5)
plt.title('Ventas totales vs. Puntuación de críticos (PS4, 2013-2016)')
plt.xlabel('Puntuación de críticos')
plt.ylabel('Ventas totales (en millones)')
plt.show()


In [ ]:
# Gráfico de dispersión para user_score vs. total_sales
plt.figure(figsize=(14,6))
plt.scatter(ps4_data_recent['user_score'], ps4_data_recent['total_sales'], color='orange', alpha=0.5)
plt.title('Ventas totales vs. Puntuación de usuarios (PS4, 2013-2016)')
plt.xlabel('Puntuación de usuarios')
plt.ylabel('Ventas totales (en millones)')
plt.show()

# Calcular la correlación entre las reseñas y las ventas
correlation_critic_recent = ps4_data_recent['critic_score'].corr(ps4_data_recent['total_sales'])
correlation_user_recent = ps4_data_recent['user_score'].corr(ps4_data_recent['total_sales'])

correlation_critic_recent, correlation_user_recent


He realizado las correcciones y regenerado los gráficos y cálculos de correlación para la plataforma PS4 en el período 2013-2016. Aquí están las observaciones basadas en los gráficos y cálculos:

In [ ]:
# Gráfico de dispersión para user_score vs. total_sales
plt.figure(figsize=(14,6))
plt.scatter(ps4_data_recent['user_score'], ps4_data_recent['total_sales'], color='orange', alpha=0.5)
plt.title('Ventas totales vs. Puntuación de usuarios (PS4)')
plt.xlabel('Puntuación de usuarios')
plt.ylabel('Ventas totales (en millones)')
plt.show()

# Calcular la correlación entre las reseñas y las ventas
correlation_critic = ps4_data_recent['critic_score'].corr(ps4_data_recent['total_sales'])
correlation_user = ps4_data_recent['user_score'].corr(ps4_data_recent['total_sales'])

correlation_critic, correlation_user


**Gráfico de Ventas Totales vs. Puntuación de Usuarios (PS4, 2013-2016)**:
- La mayoría de los puntos se agrupan en la parte inferior del gráfico, lo que indica que muchos juegos tienen ventas bajas independientemente de su puntuación.
- No hay una tendencia clara que indique que una mayor puntuación de usuario conduce a mayores ventas.

**Correlaciones**:
- **Puntuación de Críticos vs. Ventas**: Existe una correlación positiva moderada de aproximadamente 0.41 entre las ventas totales y la puntuación de los críticos para los juegos en la PS4 en el período 2013-2016. Esto sugiere que los juegos con mejores críticas tienden a tener mejores ventas.
- **Puntuación de Usuarios vs. Ventas**: La correlación entre las ventas totales y la puntuación de los usuarios es de aproximadamente -0.03, lo que indica una correlación muy débil. Esto sugiere que la puntuación de los usuarios no tiene un impacto claro en las ventas.

**Conclusiones**:
- Las reseñas de los críticos parecen tener un impacto positivo en las ventas de los juegos en la PS4 durante el período 2013-2016.
- Las reseñas de los usuarios, por otro lado, no muestran una correlación clara con las ventas. Esto puede indicar que los jugadores confían más en las reseñas de los críticos profesionales que en las opiniones de otros usuarios al decidir comprar un juego.

<a id="comparar_ventas_de_juegos_en_otras_plataformas"></a>

### Comparar ventas de juegos en otras plataformas

Ahora, consideremos un juego popular y comparemos sus ventas en diferentes plataformas. Tomaremos como ejemplo el juego "Grand Theft Auto V", que es uno de los juegos más vendidos y está disponible en múltiples plataformas.



In [ ]:
# Filtrando datos para el juego "Grand Theft Auto V" usando el dataset reciente
gta_v_sales_recent = games_recent[games_recent['name'] == 'Grand Theft Auto V'][['platform', 'total_sales']]

# Gráfico de barras para las ventas de "Grand Theft Auto V" en diferentes plataformas
plt.figure(figsize=(10,6))
sns.barplot(x='platform', y='total_sales', data=gta_v_sales_recent, palette="viridis")
plt.title('Ventas de "Grand Theft Auto V" en diferentes plataformas (2013-2016)')
plt.xlabel('Plataforma')
plt.ylabel('Ventas totales (en millones)')
plt.show()



**Observaciones**:

- **PS4**: GTA V ha tenido las mayores ventas en la plataforma PS4, superando ampliamente a las otras plataformas durante este período.
- **XOne**: La plataforma Xbox One (XOne) también muestra ventas significativas para GTA V, aunque no tan altas como en la PS4.
- **X360 y PS3**: Estas dos plataformas, aunque más antiguas, aún muestran ventas considerables para el juego, lo que indica la longevidad y popularidad de GTA V.
- **PC**: Las ventas en PC son menores en comparación con las consolas, pero aún así son notables.

**Conclusiones**:

- "Grand Theft Auto V" ha mantenido su popularidad en múltiples plataformas a lo largo de los años, especialmente en consolas más recientes como PS4 y XOne.
- La tendencia sugiere que los juegos populares, independientemente de su año de lanzamiento, pueden mantener ventas fuertes en plataformas más nuevas si son bien recibidos por la comunidad y continúan siendo relevantes.

<a id="Distribución_general_de_juegos_por_género"></a>

### Distribución general de juegos por género

Ahora, echemos un vistazo a la distribución general de los juegos por género y veamos qué géneros tienden a ser más rentables.

In [ ]:
# Filtrando datos para el juego "Grand Theft Auto V" usando el dataset reciente
gta_v_sales_recent = games_recent[games_recent['name'] == 'Grand Theft Auto V'][['platform', 'total_sales']]

# Gráfico de barras para las ventas de "Grand Theft Auto V" en diferentes plataformas
plt.figure(figsize=(10,6))
sns.barplot(x='platform', y='total_sales', data=gta_v_sales_recent, palette="viridis")
plt.title('Ventas de "Grand Theft Auto V" en diferentes plataformas (2013-2016)')
plt.xlabel('Plataforma')
plt.ylabel('Ventas totales (en millones)')
plt.show()



**Observaciones**:

- **PS4**: GTA V ha tenido las mayores ventas en la plataforma PS4, superando ampliamente a las otras plataformas durante este período.
- **XOne**: La plataforma Xbox One (XOne) también muestra ventas significativas para GTA V, aunque no tan altas como en la PS4.
- **X360 y PS3**: Estas dos plataformas, aunque más antiguas, aún muestran ventas considerables para el juego, lo que indica la longevidad y popularidad de GTA V.
- **PC**: Las ventas en PC son menores en comparación con las consolas, pero aún así son notables.

**Conclusiones**:

- "Grand Theft Auto V" ha mantenido su popularidad en múltiples plataformas a lo largo de los años, especialmente en consolas más recientes como PS4 y XOne.
- La tendencia sugiere que los juegos populares, independientemente de su año de lanzamiento, pueden mantener ventas fuertes en plataformas más nuevas si son bien recibidos por la comunidad y continúan siendo relevantes.

<a id="crea_un_perfil_de_usuario_para_cada_región"></a>

## Crea un perfil de usuario para cada región

<a id="las-cinco-plataformas-principales"></a>

### Las cinco plataformas principales. 

In [ ]:
# Determinando las cinco principales plataformas para cada región usando el dataset reciente
top_platforms_na_recent = games_recent.groupby('platform')['na_sales'].sum().sort_values(ascending=False).head(5)
top_platforms_eu_recent = games_recent.groupby('platform')['eu_sales'].sum().sort_values(ascending=False).head(5)
top_platforms_jp_recent = games_recent.groupby('platform')['jp_sales'].sum().sort_values(ascending=False).head(5)

top_platforms_na_recent, top_platforms_eu_recent, top_platforms_jp_recent



Las cinco principales plataformas en términos de ventas para cada región en el período 2013-2016 son:

**Norteamérica (NA)**:
1. PS4: 108.74 millones
2. XOne: 93.12 millones
3. X360: 81.66 millones
4. PS3: 63.50 millones
5. 3DS: 38.20 millones

**Europa (EU)**:
1. PS4: 141.09 millones
2. PS3: 67.81 millones
3. XOne: 51.59 millones
4. X360: 42.52 millones
5. 3DS: 30.96 millones

**Japón (JP)**:
1. 3DS: 67.81 millones
2. PS3: 23.35 millones
3. PSV: 18.59 millones
4. PS4: 15.96 millones
5. WiiU: 10.88 millones

**Observaciones**:

- En Norteamérica y Europa, la PS4 lidera en ventas. La Xbox One (XOne) también tiene una presencia significativa en Norteamérica.
- En Japón, la 3DS es la líder en ventas, seguida por las consolas PlayStation.

<a id="los-cinco-generos-principales-para-cada-región-y-sus-diferencias"></a>
### Los cinco géneros principales para cada región y sus diferencias


In [ ]:
# Determinando los cinco principales géneros para cada región usando el dataset reciente
top_genres_na_recent = games_recent.groupby('genre')['na_sales'].sum().sort_values(ascending=False).head(5)
top_genres_eu_recent = games_recent.groupby('genre')['eu_sales'].sum().sort_values(ascending=False).head(5)
top_genres_jp_recent = games_recent.groupby('genre')['jp_sales'].sum().sort_values(ascending=False).head(5)

top_genres_na_recent, top_genres_eu_recent, top_genres_jp_recent



Las cinco principales categorías de juegos en términos de ventas para cada región en el período 2013-2016 son:

**Norteamérica (NA)**:
1. Acción: 126.05 millones
2. Disparos: 109.74 millones
3. Deportes: 65.27 millones
4. Rol: 46.40 millones
5. Misceláneos: 27.49 millones

**Europa (EU)**:
1. Acción: 118.13 millones
2. Disparos: 87.86 millones
3. Deportes: 60.52 millones
4. Rol: 36.97 millones
5. Carreras: 20.19 millones

**Japón (JP)**:
1. Rol: 51.04 millones
2. Acción: 40.49 millones
3. Misceláneos: 9.20 millones
4. Pelea: 7.65 millones
5. Disparos: 6.61 millones

**Observaciones**:

- El género de **Acción** es popular en todas las regiones, siendo el género más vendido en Norteamérica y Europa, y el segundo más vendido en Japón.
- El género de **Disparos** es particularmente popular en Norteamérica y Europa.
- En Japón, el género de **Rol** domina las ventas, reflejando la popularidad de los juegos de rol en esta región.



<a id="si-las-clasificaciones-de-ESRB-afectan-a-las-ventas-en-regiones-individuales"></a>
###  Si las clasificaciones de ESRB afectan a las ventas en regiones individuales


In [ ]:
# Determinando las ventas totales por clasificación ESRB para cada región usando el dataset reciente
esrb_na_sales_recent = games_recent.groupby('rating')['na_sales'].sum().sort_values(ascending=False)
esrb_eu_sales_recent = games_recent.groupby('rating')['eu_sales'].sum().sort_values(ascending=False)
esrb_jp_sales_recent = games_recent.groupby('rating')['jp_sales'].sum().sort_values(ascending=False)

esrb_na_sales_recent, esrb_eu_sales_recent, esrb_jp_sales_recent



Las ventas totales basadas en las clasificaciones ESRB para cada región durante el período 2013-2016 son:

**Norteamérica (NA)**:
1. **M** (Maduro): 165.21 millones
2. **E** (Para todos): 79.05 millones
3. **E10+** (Para todos mayores de 10 años): 54.24 millones
4. **T** (Adolescente): 49.79 millones

**Europa (EU)**:
1. **M** (Maduro): 145.32 millones
2. **E** (Para todos): 83.36 millones
3. **E10+** (Para todos mayores de 10 años): 42.69 millones
4. **T** (Adolescente): 41.95 millones

**Japón (JP)**:
1. **T** (Adolescente): 20.59 millones
2. **E** (Para todos): 15.14 millones
3. **M** (Maduro): 14.11 millones
4. **E10+** (Para todos mayores de 10 años): 5.89 millones

**Observaciones**:

- En Norteamérica y Europa, los juegos clasificados como **"M" (Maduro)** dominan las ventas. Esto sugiere una preferencia por juegos orientados a un público adulto en estas regiones.
- En Japón, la clasificación **"T" (Adolescente)** tiene las ventas más altas, pero las clasificaciones **"E"** y **"M"** también tienen una presencia considerable.
- Es importante notar que no todas las clasificaciones ESRB están presentes en los datos más recientes, lo que indica que no todos los tipos de juegos se han lanzado o han sido populares en todas las regiones durante este período.


<a id="prueba-de-hipotesis"></a>
## Pruebas de Hipotésis

**Hipótesis 1:**
Las calificaciones promedio de los usuarios para las plataformas Xbox One y PC, entre los años 2013 y 2016, son las mismas.

- **Hipótesis nula (\( H_0 \))**: Las calificaciones promedio de los usuarios para Xbox One son iguales a las calificaciones promedio de los usuarios para PC.
  
- **Hipótesis alternativa (\( H_1 \))**: Las calificaciones promedio de los usuarios para Xbox One no son iguales a las calificaciones promedio de los usuarios para PC.

**Hipótesis 2:**
Las calificaciones promedio de los usuarios para los géneros de Acción y Deportes, entre los años 2013 y 2016, son diferentes.

- **Hipótesis nula (\( H_0 \))**: Las calificaciones promedio de los usuarios para el género de Acción son iguales a las calificaciones promedio de los usuarios para el género Deportes.
  
- **Hipótesis alternativa (\( H_1 \))**: Las calificaciones promedio de los usuarios para el género de Acción no son iguales a las calificaciones promedio de los usuarios para el género Deportes.

Para probar estas hipótesis, usare la prueba t de Student para muestras independientes, ya que estamos comparando las medias de dos grupos diferentes.

**Criterio para probar las hipótesis:** Utilizare la prueba t de Student para muestras independientes. Esta prueba es adecuada porque:

1. Estoy comparando las medias de dos grupos independientes.
2. La variable de interés (calificación del usuario) es continua.
3. Asumimos que las calificaciones de los usuarios tienen una distribución aproximadamente normal para cada grupo.

El nivel de significancia se quedará en alpha de 0.05. Si el p-valor resultante de la prueba t es menor que alpha, rechazaremos la hipótesis nula en favor de la hipótesis alternativa.





In [ ]:
# Filtrando los datos usando games_recent
xbox_one_scores_recent = games_recent[games_recent['platform'] == 'XOne']['user_score'].dropna()
pc_scores_recent = games_recent[games_recent['platform'] == 'PC']['user_score'].dropna()

action_scores_recent = games_recent[games_recent['genre'] == 'Action']['user_score'].dropna()
sports_scores_recent = games_recent[games_recent['genre'] == 'Sports']['user_score'].dropna()

# Nivel de significancia
alpha = 0.05

# Realizando las pruebas t
t_stat_platforms, p_value_platforms = stats.ttest_ind(xbox_one_scores_recent, pc_scores_recent)
t_stat_genres, p_value_genres = stats.ttest_ind(action_scores_recent, sports_scores_recent)

t_stat_platforms, p_value_platforms, t_stat_genres, p_value_genres



<a id="resultados-de-las-pruebas"></a>
### Resultados de las pruebas:

**Resultados de las pruebas:**

Hipótesis 1 (Plataformas Xbox One vs PC):
- Valor t: 1.2712
- p-valor: 0.2046

Dado que el p-valor (0.2046) es mayor que el nivel de significancia (0.05), no rechazamos la hipótesis nula. Esto sugiere que no hay una diferencia significativa entre las calificaciones promedio de los usuarios para las plataformas Xbox One y PC en el período de 2013 a 2016.

Hipótesis 2 (Géneros Acción vs Deportes):
- Valor t: 9.1683
- p-valor: 9.36e-19

En este caso, el p-valor es extremadamente pequeño (prácticamente cero) y mucho menor que el nivel de significancia (0.05). Por lo tanto, rechazamos la hipótesis nula. Esto indica que hay una diferencia significativa entre las calificaciones promedio de los usuarios para los géneros Acción y Deportes en el período de 2013 a 2016.


<a id="conclusiones"></a>
### Conclusiones


**Conclusiones**

1. **Calificaciones de usuarios para las plataformas Xbox One vs PC**:

   Basándome en la prueba t de Student realizada, encontramos que el p-valor para las calificaciones entre Xbox One y PC no era lo suficientemente pequeño para rechazar la hipótesis nula en el período de 2013 a 2016. Por lo tanto, no encontramos evidencia suficiente para afirmar que las calificaciones promedio de los usuarios para Xbox One y PC son diferentes durante este período. Esto sugiere que, en cuanto a calificación, los usuarios valoran de manera similar los juegos en ambas plataformas durante esos años.

2. **Calificaciones de usuarios para los géneros Acción vs Deportes**:

   El p-valor para la comparación entre los géneros Acción y Deportes resultó ser extremadamente pequeño, mucho menor que nuestro nivel de significancia alpha. Esto nos lleva a rechazar la hipótesis nula, indicando que hay diferencias significativas en las calificaciones promedio de los usuarios entre los géneros Acción y Deportes. Aunque no podemos inferir la razón exacta basándonos solo en estas pruebas, es evidente que, en términos de calificación, los usuarios no valoran de la misma manera los juegos de ambos géneros durante el período 2013-2016.



<a id="justificación"></a>

### Justificación

Dado que el p-valor (0.0001) es menor que el nivel de significancia (0.05), rechazamos la hipótesis nula. Esto indica que hay una diferencia significativa entre las calificaciones promedio de los usuarios para las plataformas Xbox One y PC.

- Hipótesis 2 (Géneros Acción vs Deportes):

**Valor t**: 0.3726  
**p-valor**: 0.7094

En este caso, el p-valor (0.7094) es mayor que el nivel de significancia (0.05), por lo que no rechazamos la hipótesis nula. Esto sugiere que no hay una diferencia significativa entre las calificaciones promedio de los usuarios para los géneros Acción y Deportes.

Criterio utilizado y justificación:
Se utilizó la Prueba t de Student para muestras independientes para comparar las medias de dos grupos diferentes.

La razón para elegir esta prueba es la siguiente:

- La Prueba t es adecuada para comparar las medias de dos grupos independientes.
- Las variables (calificaciones de los usuarios) son continuas y, por lo tanto, se pueden comparar mediante pruebas t.
- Las muestras se toman de poblaciones que se asumen con distribuciones normales. Aunque esto no se verificó explícitamente, en la práctica, con muestras grandes, la Prueba t es bastante robusta incluso si esta suposición no se cumple estrictamente.

El nivel de significancia `alpha` se estableció en 0.05, que es un valor comúnmente utilizado en muchas investigaciones. Si el p-valor resultante es menor que `alpha`, rechazamos la hipótesis nula en favor de la hipótesis alternativa.

<a id="conclusión-general"></a>
## Conclusión General

A lo largo de nuestro análisis de la industria de los videojuegos, hemos descubierto varios patrones y tendencias clave que ofrecen una visión profunda del mercado y del comportamiento de los consumidores.

1. **Evolución del Mercado**: La cantidad de juegos lanzados ha experimentado un crecimiento notable con el tiempo, aunque ha habido ciertos periodos de estancamiento. Esto refleja el crecimiento y madurez de la industria de los videojuegos y su transformación en una forma de entretenimiento dominante.

2. **Vida de las Plataformas**: Las plataformas de juegos tienen un ciclo de vida claro. Suelen emerger, crecer en popularidad, y finalmente declinar a medida que son reemplazadas por tecnologías más recientes. Esto subraya la importancia de la innovación y adaptación en esta industria.

3. **Preferencias Regionales**: Hay variaciones claras en las preferencias de juego entre las diferentes regiones. Mientras que algunas plataformas y géneros son populares globalmente, otros tienen una fuerte preferencia regional, como se observó en el mercado japonés.

4. **Influencia de las Reseñas**: Las reseñas, tanto de críticos como de usuarios, tienen un impacto en las ventas, aunque la magnitud de esta influencia puede variar. Las empresas de juegos deben ser conscientes de la importancia de la percepción y la retroalimentación de la comunidad.

5. **Clasificaciones ESRB**: Las clasificaciones ESRB, aunque no estén presentes para todos los juegos, tienen un papel en las ventas, particularmente en regiones donde estas clasificaciones son tomadas en cuenta por consumidores y minoristas.

6. **Comparación de Plataformas y Géneros**: Las calificaciones promedio de los usuarios varían entre plataformas y géneros, lo que indica diferencias en las expectativas y experiencias de los usuarios en diferentes contextos de juego.

7. **Tendencias Temporales**: La cantidad de juegos lanzados ha aumentado significativamente con el tiempo, alcanzando su punto máximo alrededor de 2008-2009, seguido de una disminución. Es probable que esta disminución se deba a factores como la saturación del mercado y la transición hacia plataformas de juegos móviles y en línea.

8. **Plataformas Populares**: A lo largo de los años, hemos visto la ascensión y caída de varias plataformas. Mientras que algunas plataformas, como PS2 y DS, tuvieron un dominio significativo en el pasado, otras como PS4 y XOne están ganando terreno en años más recientes.

9. **Géneros de Juego**: Los géneros de Acción y Deportes son predominantemente populares, lo que indica una preferencia constante entre los jugadores hacia estos géneros. Sin embargo, hay variaciones regionales en la popularidad de los géneros.


10. **Hipótesis Testadas**: Se realizó un análisis estadístico para comparar las calificaciones promedio de los usuarios entre diferentes plataformas y géneros. Se encontraron diferencias significativas en algunas comparaciones, lo que indica variaciones en las preferencias y percepciones de los usuarios.

En resumen, la industria de los videojuegos es dinámica y multifacética. Para tener éxito, las empresas deben estar atentas a las tendencias del mercado, adaptarse rápidamente a las nuevas tecnologías y ser receptivas a las opiniones y preferencias de sus usuarios. Las percepciones y comportamientos de los jugadores, como se refleja en las reseñas y las ventas, son indicadores cruciales del desempeño de un juego y ofrecen valiosas lecciones para futuros desarrollos en el campo.

**Recomendación Final**: Para los desarrolladores y comercializadores de videojuegos, es esencial prestar atención a las tendencias actuales y emergentes, las preferencias regionales y las reseñas de los usuarios. A medida que la industria continúa evolucionando, mantenerse al día con estos factores puede ser la clave para el éxito en el mercado.